### Akbota Assan
### Data Management and Ethics
### Assignment 1

# Setup

In [1]:
# Init pyspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
# Init sparksql -- Only used to format the output nicely!
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

#/air_transit_2007.csv
#/home/jovyan/air_transit_2007_sample.csv

rows = sc.textFile("/air_transit_2007.csv")
data = rows.map(lambda line: line.split(","))
# data.cache()

In [2]:
import re
import numpy as np
from scipy import stats
import pandas as pd

# Sample Query

In [3]:
sample_results = data.filter(lambda r: (r[16]=='ROC'and r[1]=='3' and r[2]=='12')) \
    .map(lambda r: (r[5] , r[17])) \
    .collect()

#format nicely!
sqlContext.createDataFrame(sample_results, ['Sched. Departure', 'Destination']).show(n=100)

+----------------+-----------+
|Sched. Departure|Destination|
+----------------+-----------+
|            1455|        CLE|
|            1840|        CLE|
|             659|        CLE|
|            1750|        EWR|
|             630|        EWR|
|            1650|        EWR|
|            1255|        EWR|
|            1025|        EWR|
|            1034|        IAD|
|            1607|        IAD|
|            1006|        ORD|
|             600|        ORD|
|            2000|        ORD|
|             628|        JFK|
|             638|        CVG|
|            1100|        CVG|
|            1141|        JFK|
|            1530|        JFK|
|            1700|        JFK|
|            1827|        CVG|
|             720|        ORD|
|            1725|        ORD|
|             800|        CLT|
|             720|        PHL|
|             600|        ATL|
|            1521|        ATL|
|            1730|        ATL|
|            1236|        ATL|
|             935|        ATL|
|       

# NA Analysis

In [4]:
for col in range(29):
    print str(col) + " " + str(data.filter(lambda r: str(r[col]) == 'NA' or str(r[col]) == "NULL").count())

0 0
1 0
2 0
3 0
4 160748
5 0
6 177927
7 0
8 0
9 0
10 0
11 177927
12 994
13 177927
14 177927
15 160748
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 0
24 0
25 0
26 0
27 0
28 0


## Q1
Compute the total number of records.

In [5]:
# Removing the first rows from the count as it contains column headers
print "Total number of records is " + str(data.map(lambda r: 1).reduce(lambda x, y: x + y) - 1)

Total number of records is 7453215


In [6]:
# Removing header 
data = data.filter(lambda r: r[28] != 'LateAircraftDelay')

# Q2

Find total number of operated flights per month, sorted by the month.

In [7]:
result = data.filter(lambda r: r[1] != "NA" and r[21] != "1" and r[21] != "NA").\
        map(lambda r: (int(r[1]), 1)).reduceByKey(lambda x,y: x + y).sortByKey().collect()

sqlContext.createDataFrame(result, ['Month', 'Total Flights Operated']).show(n=12)

+-----+----------------------+
|Month|Total Flights Operated|
+-----+----------------------+
|    1|                605782|
|    2|                540139|
|    3|                622332|
|    4|                603510|
|    5|                624768|
|    6|                612037|
|    7|                635054|
|    8|                640984|
|    9|                593680|
|   10|                622665|
|   11|                598870|
|   12|                592646|
+-----+----------------------+



# Q3

Find the plane with the highest number of flights. Each plane has a unique TailNum.

In [8]:
result = data.filter(lambda r: r[10] != "NA" and str(r[10]).isdigit() != True).\
                                                map(lambda r: (str(r[10]), 1)).\
                                                reduceByKey(lambda x, y: x + y).\
                                                sortBy(lambda x: x[1], ascending = False).first()

print "The plane with the highest number of flights is " + str(result[0]) + " with " + str(result[1]) + " flights"

The plane with the highest number of flights is N655BR with 4483 flights


# Q4

Compute the total flight time of each airplane, sorted by flight time in descending order.

In [9]:
result = data.filter(lambda r: r[10] != "NA" and str(r[10]).isdigit() != True and r[11].isnumeric() and r[11] != "NA").\
                                    map(lambda r: (str(r[10]), float(r[11]))).\
                                    reduceByKey(lambda x, y: x + y).\
                                    sortBy(lambda x: x[1], ascending = False).collect()

sqlContext.createDataFrame(result, ['Airplane', 'Total Flight Time']).show(n=20)

+--------+-----------------+
|Airplane|Total Flight Time|
+--------+-----------------+
|  N556AS|         592470.0|
|  N637JB|         296443.0|
|  N636JB|         296091.0|
|  N646JB|         293882.0|
|  N607JB|         293853.0|
|  N624JB|         292416.0|
|  N599JB|         291035.0|
|  N640JB|         290752.0|
|  N590NW|         290597.0|
|  N625JB|         290284.0|
|  N639JB|         289787.0|
|  N557UA|         289540.0|
|  N649JB|         289044.0|
|  N618JB|         288891.0|
|  N645JB|         288835.0|
|  N523JB|         287243.0|
|  N524JB|         286692.0|
|  N634JB|         286676.0|
|  N630JB|         286470.0|
|  N633JB|         286107.0|
+--------+-----------------+
only showing top 20 rows



# Q5

Find the busiest airport (in terms of number of departures + arrivals of all operated flights) for
each month.

In [10]:
# filterting out arrivals for each airport
origins = data.filter(lambda r: int(r[1]) != "NA" and str(r[16]) != "NA" and r[21] != '1' and r[21] != "NA" ).\
                                map(lambda r: ((r[1], r[16]), 1)).\
                                reduceByKey(lambda x,y: x + y)

        
# filtering out destinations for each airport
dests = data.filter(lambda r: int(r[1]) != "NA" and str(r[17]) != "NA" and r[21] != '1' and r[21] != "NA").\
                                map(lambda r: ((r[1], r[17]), 1)).\
                                reduceByKey(lambda x,y: x + y)
        

# joining arrivals and destinations for each airport
result = origins.join(dests).map(lambda x: (x[0], x[1][0] + x[1][1])).\
                            map(lambda x: (x[0][0], (x[0][1], x[1]))).\
                            reduceByKey(lambda x,y: y if int(y[1]) > int(x[1]) else x).\
                            sortBy(lambda x: int(x[0])).map(lambda x: (x[0], x[1][0], x[1][1])).\
                            collect()

sqlContext.createDataFrame(result, ['Month', 'Busiest Airport', 'Number of Arrivals and Departures']).show(n=12)

+-----+---------------+---------------------------------+
|Month|Busiest Airport|Number of Arrivals and Departures|
+-----+---------------+---------------------------------+
|    1|            ATL|                            62955|
|    2|            ATL|                            57654|
|    3|            ATL|                            66750|
|    4|            ATL|                            64953|
|    5|            ATL|                            68000|
|    6|            ATL|                            70168|
|    7|            ATL|                            72327|
|    8|            ATL|                            72850|
|    9|            ATL|                            67732|
|   10|            ATL|                            73126|
|   11|            ATL|                            68883|
|   12|            ATL|                            67674|
+-----+---------------+---------------------------------+



# Q6

Find the airline with highest average delay of each type in March 2007. Note: do not write
separate code for each error type. You should compute a single RDD where each row contains the delay type, the airline that is worst regarding that delay type, and its average delay of that type in minutes.

In [11]:
# The following RDD has a structure: ((Carrier, Delay type), (Delay value, 1))
# Summing up values of delays and 1 for each carrier
# Dividing the sum of delays by the sum of 1s to arrive at the average
# Finding an airline with worst delay for each delay type

result = data.filter(lambda r: r[8] != "NA" and r[13] != "NA" and r[14] != "NA" 
                     and r[24:29] != "NA" and r[1] != "NA").\
                    filter(lambda r: r[1] == '3').\
                    flatMap(lambda r: (((r[8], 'Arrival Delay'), (r[14], 1)), ((r[8], 'Departure Delay'), (r[15], 1)), 
                                       ((r[8], 'Carrier Delay'), (r[24], 1)), ((r[8], 'Weather Delay'), (r[25], 1)),
                                      ((r[8], 'NAS Delay'), (r[26], 1)), ((r[8], 'Security Delay'), (r[27], 1)),
                                      ((r[8], 'Late Aircraft Delay'), (r[28], 1)))).\
                                        reduceByKey(lambda x,y: (float(x[0]) + float(y[0]), int(x[1]) + int(y[1]))).\
                                        map(lambda x: (x[0], x[1][0]/x[1][1])).\
                                        map(lambda x: (x[0][1], (x[0][0], x[1]))).\
                                        reduceByKey(lambda x,y: y if y[1] > x[1] else x).\
                                        map(lambda x: (x[0], x[1][0], round(x[1][1],2))).collect()


sqlContext.createDataFrame(result, ['Delay Type', 'Worst Performing Airline', 'Average Delay']).show(n=7)

+-------------------+------------------------+-------------+
|         Delay Type|Worst Performing Airline|Average Delay|
+-------------------+------------------------+-------------+
|      Arrival Delay|                      US|        20.53|
|      Weather Delay|                      OH|         3.04|
|    Departure Delay|                      US|        19.66|
|     Security Delay|                      AS|         0.12|
|      Carrier Delay|                      EV|         9.62|
|          NAS Delay|                      CO|         7.12|
|Late Aircraft Delay|                      B6|         9.91|
+-------------------+------------------------+-------------+



# Q7

Compute median, mean, and mode of columns 12-16, 19-21 and 25-29 for the flights in the third week of 2007. Exclude the non-numeric values.

In [12]:
# Gathering values of each column into an iterable RDD list
# r[11:16] != "NA" and r[18:21] != "NA" and r[24:29] != "NA"

col_vals = data.filter(lambda r: 
                                                                            r[11]  != "NA" and 
                                                                            r[12]  != "NA" and 
                                                                            r[13]  != "NA" and 
                                                                            r[14]  != "NA" and 
                                                                            r[15]  != "NA" and 
                                                                            r[18]  != "NA" and 
                                                                            r[19]  != "NA" and 
                                                                            r[20]  != "NA" and 
                                                                            r[24]  != "NA" and 
                                                                            r[25]  != "NA" and 
                                                                            r[26]  != "NA" and 
                                                                            r[27]  != "NA" and
                                                                            r[28]  != "NA" and
                                                                            r[1]   != "NA" and
                                                                            r[2]   != "NA" and
                                                                            
                                                                            r[11].isnumeric() and 
                                                                            r[12].isnumeric() and 
                                                                            r[13].isnumeric() and 
                                                                            r[14].isnumeric() and 
                                                                            r[15].isnumeric() and 
                                                                            r[18].isnumeric() and 
                                                                            r[19].isnumeric() and 
                                                                            r[20].isnumeric() and 
                                                                            r[24].isnumeric() and 
                                                                            r[25].isnumeric() and 
                                                                            r[26].isnumeric() and 
                                                                            r[27].isnumeric() and
                                                                            r[28].isnumeric() and
                                                                            r[1].isnumeric()  and
                                                                            r[2].isnumeric()).\
                                    filter(lambda r: r[1] == '1' and int(r[2]) in range(15, 22)).\
                                    flatMap(lambda r: ((11, r[11]), 
                                                       (12, r[12]), 
                                                       (13, r[13]), 
                                                       (14, r[14]), 
                                                       (15, r[15]),
                                                       (18, r[18]), 
                                                       (19, r[19]), 
                                                       (20, r[20]), 
                                                       (24, r[24]), 
                                                       (25, r[25]),
                                                       (26, r[26]), 
                                                       (27, r[27]), 
                                                       (28, r[28]))).\
                                             groupByKey(2).collect()

In [13]:
# Gathering all values related to each column into a dictionary. 
val_dic = {}
for i in range(len(col_vals)):
    l1 = []
    for val in col_vals[i][1]:
        l1.append(float(val))
    val_dic[col_vals[i][0] + 1] = l1
    

In [14]:
# mean

for k, v in val_dic.items():
    print("Mean of column " + str(k) + " is " + str(np.mean(v)))

Mean of column 12 is 131.264174412
Mean of column 13 is 125.734685416
Mean of column 14 is 102.419548768
Mean of column 15 is 43.7846456829
Mean of column 16 is 38.255191238
Mean of column 19 is 697.689268562
Mean of column 20 is 7.52693224614
Mean of column 21 is 21.3176933974
Mean of column 25 is 9.65587534119
Mean of column 26 is 3.97885499084
Mean of column 27 is 10.9663649242
Mean of column 28 is 0.0277269115157
Mean of column 29 is 16.9902221608


In [15]:
# median

for k, v in val_dic.items():
    print("Median of column " + str(k) + " is " + str(np.median(v)))

Median of column 12 is 114.0
Median of column 13 is 109.0
Median of column 14 is 85.0
Median of column 15 is 26.0
Median of column 16 is 21.0
Median of column 19 is 556.0
Median of column 20 is 6.0
Median of column 21 is 16.0
Median of column 25 is 0.0
Median of column 26 is 0.0
Median of column 27 is 0.0
Median of column 28 is 0.0
Median of column 29 is 0.0


In [16]:
# mode

for k, v in val_dic.items():
    print("Mode of column " + str(k) + " is " + str(stats.mode(v)[0][0]))

Mode of column 12 is 80.0
Mode of column 13 is 75.0
Mode of column 14 is 59.0
Mode of column 15 is 0.0
Mode of column 16 is 0.0
Mode of column 19 is 370.0
Mode of column 20 is 4.0
Mode of column 21 is 10.0
Mode of column 25 is 0.0
Mode of column 26 is 0.0
Mode of column 27 is 0.0
Mode of column 28 is 0.0
Mode of column 29 is 0.0


# Q8

On 4 August 2007, quite a crowd gathered in Cape Canaveral to watch the launch of the Phoenix Spacecraft. The spacecraft was launched around 5:30am and most people immediately got back to the Orlando Airport (Code MCO) to return home. They could book any flight departing from 9am onwards. Find the fastest time they could arrive at any destination airport. If there were no direct flights from MCO to the destination airport, they had to book a one-stop flight, but should arrive before midnight (i.e. do not consider flights arriving after 4 August). Use scheduled times. Report the connecting airport (for one-stop flights) and sort the airports by the return time.

In [17]:
flight = data.filter(lambda r: r[1] != "NA" and r[2] != "NA" and r[5] != "NA" 
                     and r[12] != "NA" and r[16] != "NA" and r[17] != "NA")

# Convering all time values into minutes
# The following RDD is structured as follows: (Destination, (Arrival time in MOC time, Local arrival time))
# Gathering all direct flights from MOC airport that fit the criteria
# Finding the earliest arrival for each direct flight sorted by MOC arrival time


direct = flight.filter(lambda r: r[1] == '8' and  r[2] == '4' and r[16] == 'MCO' and  int(r[5]) >= 900 and
                      (int(r[5])/100)*60 + int(r[5])%100 + int(12) <= 23*60 + 59).\
                        map(lambda r: (r[17], (str((int(r[5])/100)*60 + int(r[5])%100 + int(12)), 
                                               str((int(r[7])/100)*60 + int(r[7])%100)))).\
                        reduceByKey(lambda x, y: x if int(x[1][0]) < y[1][0] else y)
        

# The following RDD is structured as follows: 
# (Origin, (Local departure time, Local arrival time, Flight duration, Destination))
# Gathering all direct flights that do not depart nor arrive at MOC airport


non_direct = flight.filter(lambda r: r[1] == '8' and  r[2] == '4' and  r[16] != 'MCO' and r[17] != 'MCO').\
                map(lambda r: (r[16], (str((int(r[5])/100)*60 + int(r[5])%100),
                                       str((int(r[7])/100)*60 + int(r[7])%100),
                                       r[12], r[17])))

In [18]:
# Joining direct flights and non-direct flights by dest = origin
# This way we ensure that the final destination is reachable through a one-time-stop from MOC
# After joining, RDD has a following structure: 
# (One-stop flight airport, ((Arrival time in MOC time, Local arrival time),(Local departure time, Local arrival time, Flight duration, Destination)))
# Filtering out cases where connecting flight's departure time is earlier than previous flight's arrival time
# Major assumption: it takes at least 1 hour for a passenger to change flights 
# Changing RDD structure to ((One-stop airport, Final destination), 
# (MOC arrival time + time between connecting flights + second flight's duration, Local arrival time))
# Filtering out flights that arrive after midnight by MOC local time
# Finding the earliest flights for each destination
# Flight arrivals are sorted by MOC local time

non_direct_shortest_flights = direct.join(non_direct).filter(lambda x:  (int(x[1][1][0]) - int(x[1][0][1])) >= 60).\
            map(lambda x: (("Stop at " + str(x[0]), x[1][1][3]), 
            (str(int(x[1][0][0]) + int(x[1][1][0]) - int(x[1][0][1]) + int(x[1][1][2])), x[1][1][1]))).\
            filter(lambda x: int(x[1][0]) <= 23*60 + 59).\
            reduceByKey(lambda x, y: x if int(x[1][0]) < int(y[1][0]) else y)

            
# Joining sorted direct and non-direct flights
# Sorting flights by MOL local time arrival 
# Final RDD structure: ((one-stop airport, final destination), (Arrival time by MOC local time, Local arrival time))

all_flights = non_direct_shortest_flights.union(direct.map(lambda x: (("Direct", x[0]), x[1]))).\
                                        sortBy(lambda x: int(x[1][0])).\
                                        map(lambda x: (str(x[0][0]) + " to " + str(x[0][1]), 
                                                              str(int(x[1][0])/60 + 1000)[2:] + 
                                                        ":" + str(int(x[1][0])%60 + 1000)[2:],  
                                                              str(int(x[1][1])/60 + 1000)[2:] + 
                                                        ":" + str(int(x[1][1])%60 + 1000)[2:])).collect()

In [19]:
sqlContext.createDataFrame(all_flights, ['Flight', 'Arrival Time MOC', 'Local Arrival Time']).show(n=100)

+------------------+----------------+------------------+
|            Flight|Arrival Time MOC|Local Arrival Time|
+------------------+----------------+------------------+
|     Direct to LEX|           09:27|             11:16|
|     Direct to GSP|           09:29|             10:48|
|     Direct to MIA|           09:57|             10:45|
|     Direct to ATL|           10:00|             11:18|
|     Direct to ISP|           10:12|             12:30|
|     Direct to PWM|           10:24|             13:27|
|     Direct to LAS|           10:27|             12:05|
|     Direct to JFK|           10:32|             12:50|
|     Direct to PHX|           10:47|             11:55|
|     Direct to CAK|           11:01|             13:06|
|     Direct to ORF|           11:07|             12:45|
|     Direct to EWR|           11:12|             13:40|
|     Direct to CMH|           11:22|             13:20|
|     Direct to HPN|           11:35|             14:00|
|     Direct to MKE|           

# Q9

Assume that a passenger wants to travel from Philadelphia International Airport (airport code: PHL) to Los Angeles International Airport (airport code: LAX), and then go back to Philadelphia (PHL). He departs PHL not earlier than 5:59 am (scheduled time), stays at least 3:01 hours in Los Angeles and then arrive at PHL not later than 11pm. Based on the "scheduled" times, find which carrier has the highest number of flights with these constraints. Limit your analysis to February 2007 and use scheduled times.

In [20]:
# My assumption here is that the return flight is not restricted to one day only, thus a person can depart on Feb 2nd and return back on Feb 14th before 11 pm

# Converting all time values into minutes
# Filterting through flights that depart in February from PHL to LAX not earlier than 5:59 am
# RDD structure -> (Airline Carrier, PHL Departure time + flight duration + 3:01 hours)

from_flight = data.filter(lambda r: r[16] == "PHL" and r[17] == "LAX" and r[1] == "2" and int(r[5]) >= 559).\
    map(lambda r: (r[8], int(r[2])*24*60 + int(r[5][:-2])*60 + int(r[5][-2:]) + int(r[12]) +181))
    

# Filterting through flights that operate in February from LAX to PHL with arrival time not later than 11 pm (PHL time)
# RDD structure -> (Airline Carrier, PHL Arrival time - flight duration)
to_flight = data.filter(lambda r: r[16] == "LAX" and r[17] == "PHL" and int(r[7]) <= 2300 and r[1] == '2').\
            map(lambda r: (r[8], int(r[2])*24*60 + int(r[7][:-2])*60 + int(r[7][-2:]) - int(r[12])))
    
# Joining flights to LAX and returning flights by Airline Carrier
# Filtering flights where return flight departure by PHL time is later than a previous flight's arrival by PHL time
# Counting the total number of eligible flights by carrier

flights_conditioned = from_flight.join(to_flight).filter(lambda x: int(x[1][1]) >= int(x[1][0])).\
                                        map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y).collect()


#sqlContext.createDataFrame(flights_conditioned, ['Airline', 'Number of Eligible Flights']).show(n=10)

print "Airline with the highest number of flights that fit the criteria is " + str(flights_conditioned[0][0])

Airline with the highest number of flights that fit the criteria is US


# Q10


Generate the `departure flights` board of the Los Angeles Airport at 12 Jan 2007 at 13:00. The board should contain flights with actual departure times between 12:00 and 14:00, sorted by scheduled departure time. The resulting table should at least contain scheduled departure time, actual departure time (if departed), airline code, and destination.



In [21]:
# Filtering out flights departing from LAX on 12 Jan 2007
flights = data.filter(lambda r: r[1] != "NA" and r[2] != "NA" 
                      and r[4] != "NA" and r[5] != "NA" and r[8] != "NA" 
                      and r[16] != "NA" and r[17] != "NA" and
                       r[1] == '1' and r[2] == '12' and r[16] == "LAX")


# Filtering out flights that have already departed 
la_board_departed = flights.filter(lambda r:  int(r[4]) >= 1200 and int(r[4]) <= 1259).\
            map(lambda r: (r[5], "Departed at " + str(r[4])[:-2] + ":" + str(r[4])[-2:], r[8], r[17]))
    
# Filtering out flights that are yet to depart
la_board_scheduled = flights.filter(lambda r:  int(r[4]) >= 1300 and int(r[4]) <= 1400).\
            map(lambda r: (r[5], "Scheduled", r[8], r[17]))

# Gathering flight info on both scheduled and departed flights
# Sorting flights by scheduled departure time
la_board_full = la_board_departed.union(la_board_scheduled).sortBy(lambda x: x[0]).\
                map(lambda x: (str(x[0][:-2]) + ":" + str(x[0][-2:]), x[1], x[2], x[3])).collect()


In [22]:
labels = ['Scheduled Departure', 'Status', 'Airline', 'Destination']
pd.DataFrame.from_records(la_board_full, columns=labels)

,Scheduled Departure,Status,Airline,Destination
0,11:00,Departed at 12:08,UA,ORD
1,11:40,Departed at 12:18,OO,OXR
2,11:40,Departed at 12:02,AA,DFW
3,11:42,Departed at 12:06,DL,CVG
4,11:45,Departed at 12:10,WN,SLC
5,11:55,Departed at 12:52,MQ,SJC
6,11:55,Departed at 12:42,AA,LAS
7,11:57,Departed at 12:16,OO,PSP
8,12:00,Departed at 12:56,WN,OAK
9,12:00,Departed at 12:13,CO,IAH
